# Potencial RCI 60

Este cuaderno elabora el ensayo de potencial de RCI 60 de clientes con precalificaciones Horus en compra saldo y Liquidez. Se quiere evaluar qué clientes necesitarían llegar a un RCI de hasta 60 para tener una consolidación parcial o total de sus deudas externas. Además, se hace una regresión lineal del aumento de RCI respecto del salario para simplificar la relación.

In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append(r"C:\Users\mariajose_chinchilla\Documents\GitHub\proyectos_varios\codigos")
from utils import ingresos_financieros, ingreso_fin_entre_fechas, meses_entre_fechas, calcular_cuota

In [2]:
compra_saldo = pd.read_csv("../db/compra_saldo_existentes.csv")
liquidez = pd.read_csv("../db/liquidez_existentes.csv")

## Limpieza de datos

In [ ]:
compra_saldo.drop_duplicates(subset="DPI", inplace=True)
compra_saldo.columns

In [ ]:
compra_saldo = compra_saldo[["Iniciativa", "No_Precalificacion", "DPI", "TIPO_CLIENTE",
                             "RCI_Interno_Inicial", "RCI_Global_Inicial",
                             "Ingreso_Validado", "Monto_Maximo_a_Ofertar", 
                             "Cantidad_Creditos_Externos_a_consolidar", 
                             "Saldo_Total_de_Creditos_Externo_a_consolidar", 
                             "Cuota_Total_de_Creditos_Externos_a_consolidar", 
                             "PATRONO", "SEGMENTO", "NUMERO_RENGLON",
                             "CONSOLIDA_BANRURAL", "CLIENTE_BANRURAL",
                             "CLIENTE_INDUSTRIAL", "CONSOLIDA_INDUSTRIAL",
                             "Color_Resultado",
                             'scoreCredito', 'scoreCreditoPricing']]

compra_saldo.info()

In [ ]:
compra_saldo.describe()

In [ ]:
liquidez.drop_duplicates(subset="DPI", inplace=True)
liquidez.columns

In [ ]:
liquidez = liquidez[["Iniciativa", "No_Precalificacion", "IngresoValidado",
                     "DPI", "SEGMENTO", "TIPO_CLIENTE", "RCI_Interno_Inicial",
                     "RCI_Global_Inicial", "Monto_Maximo_a_Ofertar",
                    "PATRONO", "CLIENTE_BANRURAL", "CLIENTE_BANRURAL"]]

liquidez.info()

In [ ]:
liquidez.describe()

## Descripción de la muestra

In [ ]:
clientes_liquidez = liquidez["DPI"].unique().tolist()
clientes_compra = compra_saldo["DPI"].unique().tolist()
compartidos = [x for x in clientes_liquidez if x in clientes_compra]
clientes_distintos_total = len(clientes_liquidez) + len(clientes_compra) - len(compartidos)

print(f"Se trabajaron {len(clientes_liquidez)} clientes de liquidez, {len(clientes_compra)} clientes de compra_saldo.")
print(f"Esto representa, entre las dos iniciativas, {clientes_distintos_total} clientes distintos.")
print(f"Esto representa {len(clientes_liquidez) + len(clientes_compra) - clientes_distintos_total} clientes repetidos")

In [ ]:
# Genera codigo latex para las tablas
latex_compra = compra_saldo[['RCI_Interno_Inicial', 'Ingreso_Validado',
    'Cantidad_Creditos_Externos_a_consolidar',
       'Saldo_Total_de_Creditos_Externo_a_consolidar',
       'Cuota_Total_de_Creditos_Externos_a_consolidar']].describe().to_latex()

latex_liquidez = liquidez[['RCI_Interno_Inicial', 'RCI_Global_Inicial', 
                           'Monto_Maximo_a_Ofertar']].describe().to_latex()

dic_latex = {"latex_compra": latex_compra, "latex_liquidez": latex_liquidez}
for cod in dic_latex.keys():
    with open(f"../db/LaTeX/{cod}.txt", "w") as f:
        f.write(dic_latex.get(cod))
        print(f"Se guardó el txt del LaTeX para {cod}")

In [11]:
compra_saldo = compra_saldo[~compra_saldo["DPI"].isin(compartidos)]
liquidez = liquidez[~liquidez["DPI"].isin(compartidos)]

In [ ]:
(len(compra_saldo[compra_saldo["TIPO_CLIENTE"] == "CLIENTE REACTIVADO"]) + len(liquidez[liquidez["TIPO_CLIENTE"] == "CLIENTE REACTIVADO"])) / (len(compra_saldo) + len(liquidez))

## Simulación

In [13]:
plazo_simulacion = 144

### Compra saldo

In [14]:
def tasa_general(segmento, tipo_pago, tipo_cliente):
    tasa = None
    # privado remesa
    if segmento == "PRIVADO" and tipo_pago not in ["Pagos Directos a Caja Privadas"]:
        if tipo_cliente == "CLIENTE EXISTENTE":
            tasa = 18.34
        if tipo_cliente == "CLIENTE NUEVO":
            tasa = 15.62
        if tipo_cliente == "CLIENTE REACTIVADO":
            tasa = 16.05
    # privado voluntario
    if segmento == "PRIVADO" and tipo_pago not in ["Pagos Directos a Caja Privadas"]:
        if tipo_cliente == "CLIENTE EXISTENTE":
            tasa = 24.83
        if tipo_cliente == "CLIENTE NUEVO":
            tasa = 26.61
        if tipo_cliente == "CLIENTE REACTIVADO":
            tasa = 26.25
    if segmento != "PRIVADO" and tipo_pago not in ["Pagos por Caja Entidades Descentralizadas", "Pagos Directos a Caja Independientes"]:
        if tipo_cliente == "CLIENTE EXISTENTE":
            tasa = 18.37
        if tipo_cliente == "CLIENTE NUEVO":
            tasa = 15.98
        if tipo_cliente == "CLIENTE REACTIVADO":
            tasa = 15.62
    if segmento != "PRIVADO" and tipo_pago  in ["Pagos por Caja Entidades Descentralizadas", "Pagos Directos a Caja Independientes"]:
        if tipo_cliente == "CLIENTE EXISTENTE":
            tasa = 24.47
        if tipo_cliente == "CLIENTE NUEVO":
            tasa = 27.33
        if tipo_cliente == "CLIENTE REACTIVADO":
            tasa = 28
    return tasa
    
compra_saldo["TASA GENERAL"] = compra_saldo.apply(lambda x: tasa_general(x["SEGMENTO"], x["PATRONO"], x["TIPO_CLIENTE"]), axis=1)
compra_saldo["NUEVA CUOTA CONSOL TOTAL"] = compra_saldo.apply(lambda x: calcular_cuota(x["Saldo_Total_de_Creditos_Externo_a_consolidar"],
                                                              x["TASA GENERAL"]/1200, plazo_simulacion), axis=1)
compra_saldo["NUEVO RCI"] = compra_saldo["RCI_Interno_Inicial"] + 100*(compra_saldo["NUEVA CUOTA CONSOL TOTAL"] / compra_saldo["Ingreso_Validado"])
compra_saldo["CONSOLIDA TOTAL"] = np.where(compra_saldo["NUEVO RCI"] <= 60, 1, 0)

In [ ]:
prop1 = len(compra_saldo[compra_saldo["NUEVO RCI"] <= 60]) 
cantidad_no_consolidacion = len(compra_saldo) - prop1
print(f"El {prop1 } de los clientes de precalificación de compra saldo Horus puede consolidar")
print("por completo su deuda y usar la inciativa RCI 60.")
print(f"No se pueden consolidar a {cantidad_no_consolidacion} clientes")

In [ ]:
viables_compra_saldo = compra_saldo[compra_saldo["CONSOLIDA TOTAL"] == 1]

# Generales
desembolso_bruto_potencial = viables_compra_saldo["Saldo_Total_de_Creditos_Externo_a_consolidar"].sum()
desembolso_bruto_promedio = viables_compra_saldo["Saldo_Total_de_Creditos_Externo_a_consolidar"].mean()
tpp_esperada = sum(viables_compra_saldo["TASA GENERAL"] * viables_compra_saldo["Saldo_Total_de_Creditos_Externo_a_consolidar"]) / sum(viables_compra_saldo["Saldo_Total_de_Creditos_Externo_a_consolidar"])
nuevo_rci_prom = viables_compra_saldo["NUEVO RCI"].mean()

# PV
mascara_voluntario = viables_compra_saldo["PATRONO"].isin(["Pagos por Caja Entidades Descentralizadas", "Pagos Directos a Caja Independientes", "Pagos Directos a Caja Privadas"])
mascara_existente = viables_compra_saldo["TIPO_CLIENTE"] == "CLIENTE EXISTENTE"
mascara_reactivado = viables_compra_saldo["TIPO_CLIENTE"] == "CLIENTE REACTIVADO"
mascara_nuevo = viables_compra_saldo["TIPO_CLIENTE"] == "CLIENTE NUEVO"


desembolso_bruto_potencial_vol = viables_compra_saldo.loc[mascara_voluntario]["Saldo_Total_de_Creditos_Externo_a_consolidar"].sum()
tpp_esperada_vol = sum(viables_compra_saldo.loc[mascara_voluntario]["TASA GENERAL"] * viables_compra_saldo.loc[mascara_voluntario]["Saldo_Total_de_Creditos_Externo_a_consolidar"]) / sum(viables_compra_saldo.loc[mascara_voluntario]["Saldo_Total_de_Creditos_Externo_a_consolidar"])
rci_exis_pv = viables_compra_saldo.loc[mascara_voluntario & mascara_existente]["NUEVO RCI"].mean()
rci_nv_pv = viables_compra_saldo.loc[mascara_voluntario & mascara_nuevo]["NUEVO RCI"].mean()
rci_reac_pv = viables_compra_saldo.loc[mascara_voluntario & mascara_reactivado]["NUEVO RCI"].mean()


# Remesa
desembolso_bruto_potencial_rem = viables_compra_saldo.loc[~mascara_voluntario]["Saldo_Total_de_Creditos_Externo_a_consolidar"].sum()
tpp_esperada_rem = sum(viables_compra_saldo.loc[~mascara_voluntario]["TASA GENERAL"] * viables_compra_saldo.loc[~mascara_voluntario]["Saldo_Total_de_Creditos_Externo_a_consolidar"]) / sum(viables_compra_saldo.loc[~mascara_voluntario]["Saldo_Total_de_Creditos_Externo_a_consolidar"])
rci_exis_r = viables_compra_saldo.loc[~mascara_voluntario & mascara_existente]["NUEVO RCI"].mean()
rci_nv_r = viables_compra_saldo.loc[~mascara_voluntario & mascara_nuevo]["NUEVO RCI"].mean()
rci_reac_r = viables_compra_saldo.loc[~mascara_voluntario & mascara_reactivado]["NUEVO RCI"].mean()

print(f"El potencial de desembolso bruto (solamente consolidaciones) es de {desembolso_bruto_potencial / 10**6} millones, promedio de {desembolso_bruto_promedio}")
#print(f"que se divide en {desembolso_bruto_potencial_vol} para pago voluntario y {desembolso_bruto_potencial_rem} en remesa.")
print(f"La TPP general esperada es de {tpp_esperada_rem}.")
print(f"El nuevo RCI  esperado es de {nuevo_rci_prom}")

In [17]:
cod_via_compra = viables_compra_saldo[['NUEVO RCI', 
                                       "TASA GENERAL",
       'Saldo_Total_de_Creditos_Externo_a_consolidar',
       'Cuota_Total_de_Creditos_Externos_a_consolidar', "NUEVA CUOTA CONSOL TOTAL"]].describe().to_latex()

with open("../db/LaTeX/Viables_compra.txt", "w") as f:
    f.write(cod_via_compra)

### Liquidez

In [ ]:
prop_existente = len(liquidez[liquidez["TIPO_CLIENTE"] == "Cliente Existente"]) / len(liquidez)
print(f"El {round(prop_existente * 100, 2)}% de los clientes son clientes existentes.")

In [19]:
def tasa_liquidez(segmento, tipo_pago, tipo_cliente):
    tasa = -1
    # privado remesa
    if segmento == "PRIVADO" and tipo_pago not in ["Pagos Directos a Caja Privadas"]:
        if tipo_cliente == "CLIENTE EXISTENTE":
            tasa = 20.61
        if tipo_cliente == "CLIENTE NUEVO":
            tasa = 17
        if tipo_cliente == "CLIENTE REACTIVADO":
            tasa = 16.93
    # privado voluntario
    if segmento == "PRIVADO" and tipo_pago not in ["Pagos Directos a Caja Privadas"]:
        if tipo_cliente == "CLIENTE EXISTENTE":
            tasa = 26.36
        if tipo_cliente == "CLIENTE NUEVO":
            tasa = 26.27
        if tipo_cliente == "CLIENTE REACTIVADO":
            tasa = 27.72
    # no privado
    # remesa
    if segmento != "PRIVADO" and tipo_pago not in ["Pagos por Caja Entidades Descentralizadas", "Pagos Directos a Caja Independientes"]:
        if tipo_cliente == "CLIENTE EXISTENTE":
            tasa = 19.23
        if tipo_cliente == "CLIENTE NUEVO":
            tasa = 16.71
        if tipo_cliente == "CLIENTE REACTIVADO":
            tasa = 16.62
    # pv
    if segmento != "PRIVADO" and tipo_pago  in ["Pagos por Caja Entidades Descentralizadas", "Pagos Directos a Caja Independientes"]:
        if tipo_cliente == "CLIENTE EXISTENTE":
            tasa = 19
        if tipo_cliente == "CLIENTE NUEVO":
            tasa = 25.61
        if tipo_cliente == "CLIENTE REACTIVADO":
            tasa = 23.91
    return tasa

liquidez["TIPO_CLIENTE"] = liquidez["TIPO_CLIENTE"].astype(str).str.replace("Cliente Existente", "CLIENTE EXISTENTE")
liquidez["TIPO_CLIENTE"] = liquidez["TIPO_CLIENTE"].astype(str).str.replace("Cliente Nuevo", "CLIENTE NUEVO")
liquidez["TIPO_CLIENTE"] = liquidez["TIPO_CLIENTE"].astype(str).str.replace("Cliente Reactivado", "CLIENTE REACTIVADO")


monto_simulacion = 50000
liquidez["TASA GENERAL"] = liquidez.apply(lambda x: tasa_liquidez(x["SEGMENTO"], x["PATRONO"], x["TIPO_CLIENTE"]), axis=1)
liquidez["NUEVA CUOTA"] = liquidez.apply(lambda x: calcular_cuota(monto_simulacion, x["TASA GENERAL"]/1200, 120), axis=1)
liquidez["NUEVO RCI"] = liquidez["RCI_Interno_Inicial"] + 100 * (liquidez["NUEVA CUOTA"] / liquidez["IngresoValidado"])

In [ ]:
prop2 = len(liquidez[liquidez["NUEVO RCI"] <= 60]) 
no_liquidez = len(liquidez) - prop2
print(f"El {prop2 } de los clientes de precalificación de liquidez Horus puede usar la iniciativa RCI60")
print(f"{no_liquidez} no pueden ser atendidos en liquidez")

In [ ]:
viables_liquidez = liquidez[liquidez["NUEVO RCI"] <= 60]

# Generales
desembolso_bruto_potencial_liquidez = monto_simulacion * len(viables_liquidez)
desembolso_bruto_promedio_liquidez = monto_simulacion
tpp_esperada_liquidez = sum(viables_liquidez["TASA GENERAL"] * monto_simulacion) / desembolso_bruto_potencial_liquidez
nuevo_rci_prom_liquidez = viables_liquidez["NUEVO RCI"].mean()

# PV
mascara_voluntario_liquidez = viables_liquidez["PATRONO"].isin(["Pagos por Caja Entidades Descentralizadas", "Pagos Directos a Caja Independientes", "Pagos Directos a Caja Privadas"])
mascara_existente_liquidez = viables_liquidez["TIPO_CLIENTE"] == "CLIENTE EXISTENTE"
mascara_reactivado_liquidez = viables_liquidez["TIPO_CLIENTE"] == "CLIENTE REACTIVADO"
mascara_nuevo_liquidez = viables_liquidez["TIPO_CLIENTE"] == "CLIENTE NUEVO"

try:
    desembolso_bruto_potencial_vol_liquidez = len(viables_liquidez.loc[mascara_voluntario_liquidez]) * monto_simulacion
    rci_exis_pv_liquidez = viables_liquidez.loc[mascara_voluntario_liquidez & mascara_existente_liquidez]["NUEVO RCI"].mean()
    rci_nv_pv_liquidez = viables_liquidez.loc[mascara_voluntario_liquidez & mascara_nuevo_liquidez]["NUEVO RCI"].mean()
    rci_reac_pv_liquidez = viables_liquidez.loc[mascara_voluntario_liquidez & mascara_reactivado_liquidez]["NUEVO RCI"].mean()
    tpp_esperada_vol_liquidez = sum(viables_liquidez.loc[mascara_voluntario_liquidez]["TASA GENERAL"] * monto_simulacion) / desembolso_bruto_potencial_vol_liquidez
except:
    pass 

# Remesa
desembolso_bruto_potencial_rem_liquidez = len(viables_liquidez.loc[~mascara_voluntario_liquidez]) * monto_simulacion
desembolso_bruto_potencial_rem_liquidez
rci_exis_r_liquidez = viables_liquidez.loc[~mascara_voluntario_liquidez & mascara_existente_liquidez]["NUEVO RCI"].mean()
rci_nv_r_liquidez = viables_liquidez.loc[~mascara_voluntario_liquidez & mascara_nuevo_liquidez]["NUEVO RCI"].mean()
rci_reac_r_liquidez = viables_liquidez.loc[~mascara_voluntario_liquidez & mascara_reactivado_liquidez]["NUEVO RCI"].mean()
tpp_esperada_rem_liquidez = sum(viables_liquidez.loc[~mascara_voluntario_liquidez]["TASA GENERAL"] * monto_simulacion) / desembolso_bruto_potencial_rem_liquidez

try:
    print(f"El potencial de desembolso bruto es de {desembolso_bruto_potencial_liquidez / 10**6} millones, promedio de {desembolso_bruto_promedio_liquidez}")
    #print(f"que se divide en {desembolso_bruto_potencial_vol_liquidez} para pago voluntario y {desembolso_bruto_potencial_rem_liquidez} en remesa.")
    print(f"El nuevo RCI esperado es de {nuevo_rci_prom_liquidez}")
    #print(f"{rci_reac_pv_liquidez} para CR. En el caso de Remesa ")
    #print(f"tenemos {rci_exis_r_liquidez} para CE, {rci_nv_r_liquidez} para CN y {rci_reac_r_liquidez} para CR")
    print(f"La TPP de Remesa es de {tpp_esperada_rem_liquidez}.")
    #print(f"y de {tpp_esperada_vol_liquidez} en PV.")
except:
    pass


In [ ]:
len(viables_compra_saldo)

### Regresión lineal

In [23]:
import matplotlib.pyplot as plt
from scipy import stats

In [24]:
# compra saldo
compra_saldo_temp = compra_saldo[(~compra_saldo["NUEVO RCI"].isna()) & (compra_saldo["NUEVO RCI"] > 0)]
compra_saldo_temp["AUMENTO RCI"] = compra_saldo_temp["NUEVO RCI"] - compra_saldo_temp["RCI_Interno_Inicial"]
X = compra_saldo_temp["AUMENTO RCI"]
Y = compra_saldo_temp["Saldo_Total_de_Creditos_Externo_a_consolidar"]

pendiente, intercepto, t, p, desviacion = stats.linregress(X, Y)
regresion = pendiente * X + intercepto

In [ ]:
plt.scatter(X, Y, color='blue', label='Datos reales')
plt.plot(X, regresion, color='red', label='Línea de regresión')
plt.title('Saldo a consolidar según aumento de RCI')
plt.xlabel('Aumento de RCI')
plt.ylabel('Saldo a consolidar')
equation = f'y = {pendiente:.2f}x + {intercepto:1f}'
plt.text(25, 260000, equation, fontsize=8, color='black')  
plt.legend()
plt.show()

## Exportar bases para comerciales

In [59]:
df_compra_comerciales = viables_compra_saldo[["DPI", "RCI_Global_Inicial",
                                              "Monto_Maximo_a_Ofertar"]]

df_compra_comerciales["Iniciativa"] = "Compra Saldo"

C:\Users\mariajose_chinchilla\AppData\Local\Temp\ipykernel_8420\3281893883.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_compra_comerciales["Iniciativa"] = "Compra Saldo"


In [60]:
df_liquidez_comerciales = viables_liquidez[["DPI", "RCI_Global_Inicial", 
                                            "Monto_Maximo_a_Ofertar"]]
                                            
df_liquidez_comerciales["Iniciativa"] = "Liquidez"

C:\Users\mariajose_chinchilla\AppData\Local\Temp\ipykernel_8420\1404961587.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_liquidez_comerciales["Iniciativa"] = "Liquidez"


In [64]:
df_final = pd.concat([df_liquidez_comerciales, df_compra_comerciales])
df_final.to_csv("../db/Final_RCI60.csv")